In [1]:
#import subprocess
#subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'light_NN.ipynb'])

import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier 

from __future__ import print_function
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

import copy

Using TensorFlow backend.


## Preprocessing

In [2]:

train= pd.read_csv("../data/train.csv")
test= pd.read_csv("../data/test.csv")
#train.head(3)

In [3]:
train= pd.read_csv("../data/train.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
test= pd.read_csv("../data/test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
combine1 = [train]

for train in combine1: 
        train['Salutation'] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
for train in combine1: 
        train['Salutation'] = train['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
        train['Salutation'] = train['Salutation'].replace('Mlle', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Ms', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Mme', 'Mrs')
        del train['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 

for train in combine1: 
        train['Salutation'] = train['Salutation'].map(Salutation_mapping) 
        train['Salutation'] = train['Salutation'].fillna(0)
        
for train in combine1: 
        train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
        train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
        train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'], np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 
        train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
        del train['Ticket'] 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)

for train in combine1: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)



In [4]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett
0,1,0,3,0,22.0,1,0,7.2500,0.0,1.0,3,9,0
1,2,1,1,1,38.0,1,0,71.2833,1.0,3.0,0,8,1
2,3,1,3,1,26.0,0,0,7.9250,0.0,2.0,3,16,0
3,4,1,1,1,35.0,1,0,53.1000,0.0,3.0,1,6,1
4,5,0,3,0,35.0,0,0,8.0500,0.0,1.0,3,6,0


## Preprocessing　wish NN

In [5]:
train_drop_age = copy.deepcopy(train)

In [6]:
train_drop_age.dropna(inplace=True)

In [7]:
for_age_train = train_drop_age.drop(['Survived'], axis=1).dropna(axis=0)
X_train_age = train_drop_age.drop('Age', axis=1)
y_train_age = train_drop_age['Age']

In [8]:
# create model
tmodel = Sequential()
tmodel.add(Dense(input_dim=X_train_age.shape[1], units=128,
                 kernel_initializer='normal', bias_initializer='zeros'))
tmodel.add(Activation('relu'))

for i in range(0, 8):
    tmodel.add(Dense(units=64, kernel_initializer='normal',
                     bias_initializer='zeros'))
    tmodel.add(Activation('relu'))
    tmodel.add(Dropout(.25))

tmodel.add(Dense(units=1))
tmodel.add(Activation('linear'))

tmodel.compile(loss='mean_squared_error', optimizer='adam')

In [9]:
%%time
tmodel.fit(X_train_age.values, y_train_age.values, epochs=1000, verbose=2)

Epoch 1/1000
 - 7s - loss: 941.5531
Epoch 2/1000
 - 0s - loss: 492.3951
Epoch 3/1000
 - 0s - loss: 459.9361
Epoch 4/1000
 - 0s - loss: 419.7690
Epoch 5/1000
 - 0s - loss: 424.0694
Epoch 6/1000
 - 0s - loss: 439.1907
Epoch 7/1000
 - 0s - loss: 393.3612
Epoch 8/1000
 - 0s - loss: 459.4950
Epoch 9/1000
 - 0s - loss: 354.8121
Epoch 10/1000
 - 0s - loss: 320.8493
Epoch 11/1000
 - 0s - loss: 302.6366
Epoch 12/1000
 - 0s - loss: 283.2532
Epoch 13/1000
 - 0s - loss: 290.2552
Epoch 14/1000
 - 0s - loss: 279.0844
Epoch 15/1000
 - 0s - loss: 267.2482
Epoch 16/1000
 - 0s - loss: 258.5429
Epoch 17/1000
 - 0s - loss: 245.8076
Epoch 18/1000
 - 0s - loss: 248.0115
Epoch 19/1000
 - 0s - loss: 242.3790
Epoch 20/1000
 - 0s - loss: 237.0810
Epoch 21/1000
 - 0s - loss: 238.6305
Epoch 22/1000
 - 0s - loss: 236.4899
Epoch 23/1000
 - 0s - loss: 236.2541
Epoch 24/1000
 - 0s - loss: 238.3910
Epoch 25/1000
 - 0s - loss: 231.0193
Epoch 26/1000
 - 0s - loss: 241.8254
Epoch 27/1000
 - 0s - loss: 234.6132
Epoch 28/1

In [10]:
to_pred = train.loc[train['Age'].isnull()].drop(['Age'], axis=1)

In [11]:
to_pred.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett
5,6,0,3,0,0,0,8.4583,2.0,1.0,3,6,0
17,18,1,2,0,0,0,13.0000,0.0,1.0,2,6,0
19,20,1,3,1,0,0,7.2250,1.0,3.0,2,4,0
26,27,0,3,0,0,0,7.2250,1.0,1.0,2,4,0
28,29,1,3,1,0,0,7.8792,2.0,2.0,3,6,0


In [12]:
p = tmodel.predict(to_pred.values)
train['Age'].loc[train['Age'].isnull()] = p.reshape(177,)
train['Age']=np.round(train['Age'])

/home/iwailab-ml/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
for train in combine1:
    train['IsAlone'] = 0
    train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [14]:
train["Embarked"].fillna(train.Embarked.mean(), inplace=True)
train_data = train.values
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ


In [15]:
test= pd.read_csv("../data/test.csv")
test= pd.read_csv("../data/test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

In [16]:
test["Age"].fillna(train.Age.mean(), inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

combine = [test]
for test in combine:
    test['Salutation'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for test in combine:
    test['Salutation'] = test['Salutation'].replace(['Lady', 'Countess','Capt', 'Col',\
         'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    test['Salutation'] = test['Salutation'].replace('Mlle', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Ms', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Mme', 'Mrs')
    del test['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for test in combine:
    test['Salutation'] = test['Salutation'].map(Salutation_mapping)
    test['Salutation'] = test['Salutation'].fillna(0)

for test in combine:
        test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
        test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x))
        test['Ticket_Lett'] = np.where((test['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), test['Ticket_Lett'],
                                   np.where((test['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0', '0'))
        test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
        del test['Ticket']
test['Ticket_Lett']=test['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

for test in combine:
        test['Cabin_Lett'] = test['Cabin'].apply(lambda x: str(x)[0])
        test['Cabin_Lett'] = test['Cabin_Lett'].apply(lambda x: str(x))
        test['Cabin_Lett'] = np.where((test['Cabin_Lett']).isin(['T', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']),test['Cabin_Lett'],
                                   np.where((test['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0','0'))        
        del test['Cabin']
test['Cabin_Lett']=test['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1).replace("G",1) 

test["FamilySize"] = train["SibSp"] + train["Parch"] + 1

for test in combine:
    test['IsAlone'] = 0
    test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1
    
test_data = test.values
xs_test = test_data[:, 1:]


In [17]:
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=51, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

random_forest.fit(xs, y)
Y_pred = random_forest.predict(xs_test)


import csv
with open("087081.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test_data[:,0].astype(int), Y_pred.astype(int)):
        writer.writerow([pid, survived])

## light GBM

In [59]:
ex= pd.read_csv("087081.csv")
test["Survived"]=ex["Survived"]
train=train.append(test, ignore_index=True)

In [60]:
train=train.ix[:,[7,12,8,10,0,11,6,4,2,9,14,13,1,3,5]]
train_data = train.values
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ
test_data = test.values
xs_test = test_data[:, 1:]

In [61]:
xs_test

array([[ 3.        ,  0.        , 34.5       , ...,  2.        ,
         0.        ,  0.        ],
       [ 3.        ,  1.        , 47.        , ...,  2.        ,
         0.        ,  0.        ],
       [ 2.        ,  0.        , 62.        , ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [ 3.        ,  0.        , 38.5       , ...,  1.        ,
         1.        ,  0.        ],
       [ 3.        ,  0.        , 29.42536476, ...,  3.        ,
         0.        ,  0.        ],
       [ 3.        ,  0.        , 29.42536476, ...,  3.        ,
         0.        ,  1.        ]])

# light GBM

In [62]:
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(xs,y,test_size=0.008,random_state=0)

lgb_train = lgb.Dataset(np.array(X_train),np.array(y_train))
lgb_eval = lgb.Dataset(np.array(X_test),np.array(y_test),reference=lgb_train)
 

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 200,
    'learning_rate': 0.003,
    'num_iterations':100,
    'feature_fraction': 0.52,
    'bagging_fraction': 0.79,
    'bagging_freq': 7,
    'verbose':0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10000)
X_pred= gbm.predict(np.array(xs_test), num_iteration=gbm.best_iteration)

[1]	valid_0's l2: 0.209728
Training until validation scores don't improve for 10000 rounds.
[2]	valid_0's l2: 0.208602
[3]	valid_0's l2: 0.207567
[4]	valid_0's l2: 0.206637
[5]	valid_0's l2: 0.205533
[6]	valid_0's l2: 0.204854
[7]	valid_0's l2: 0.204289
[8]	valid_0's l2: 0.203289
[9]	valid_0's l2: 0.202251
[10]	valid_0's l2: 0.201606
[11]	valid_0's l2: 0.20052
[12]	valid_0's l2: 0.1995
[13]	valid_0's l2: 0.198536
[14]	valid_0's l2: 0.197455
[15]	valid_0's l2: 0.196924
[16]	valid_0's l2: 0.196047
[17]	valid_0's l2: 0.195108
[18]	valid_0's l2: 0.194399
[19]	valid_0's l2: 0.193394
[20]	valid_0's l2: 0.192866
[21]	valid_0's l2: 0.1924
[22]	valid_0's l2: 0.19183
[23]	valid_0's l2: 0.190821
[24]	valid_0's l2: 0.189767
[25]	valid_0's l2: 0.189136
[26]	valid_0's l2: 0.188258
[27]	valid_0's l2: 0.187349
[28]	valid_0's l2: 0.186332
[29]	valid_0's l2: 0.185675
[30]	valid_0's l2: 0.18498
[31]	valid_0's l2: 0.184558
[32]	valid_0's l2: 0.183728
[33]	valid_0's l2: 0.182852
[34]	valid_0's l2: 0.182031

/home/iwailab-ml/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


## save_data

In [63]:
for i in range(418):
    if X_pred[i]>=0.51:
        X_pred[i]=1
    else:
        X_pred[i]=0

import csv
with open("Predict_result_data.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test_data[:,0].astype(int), X_pred.astype(int)):
    #for pid, survived in zip(test_data[:,0].astype(int), X_pred):
        writer.writerow([pid, survived])

In [64]:
ex= pd.read_csv("087081.csv")
tre= pd.read_csv("Predict_result_data.csv")
for i in range(417):
    if ex['Survived'][i]-tre['Survived'][i]!=0:
        print(ex['PassengerId'][i])

896
900
906
907
910
913
914
916
924
925
926
933
935
941
942
944
945
951
955
956
957
958
961
962
966
969
971
972
978
979
981
982
984
990
992
996
1004
1005
1006
1009
1010
1011
1012
1017
1019
1030
1033
1036
1038
1048
1049
1050
1051
1053
1054
1057
1060
1067
1068
1070
1073
1074
1076
1078
1084
1086
1088
1089
1092
1093
1094
1095
1100
1105
1110
1112
1114
1116
1117
1123
1130
1131
1132
1133
1134
1137
1138
1140
1144
1150
1154
1155
1165
1167
1173
1175
1176
1179
1188
1197
1199
1206
1207
1216
1222
1225
1235
1237
1241
1246
1248
1251
1253
1254
1259
1260
1266
1267
1275
1277
1282
1283
1284
1289
1292
1294
1296
1306
